<a href="https://colab.research.google.com/github/ohudalraddadi-hub/ai-arabic-text-detection/blob/main/Phase_3_Stylometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup & imports

In [ ]:
# --- Optional: installs for a fresh Colab ---
%pip install camel-tools regex numpy pandas scipy scikit-learn
%pip install -U "transformers==4.36.2" "sentence-transformers==2.5.1" torch

  Using cached transformers-4.36.2-py3-none-any.whl.metadata (126 kB)
  Using cached sentence_transformers-2.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached torch-2.9.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached tokenizers-0.15.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cublas_cu12-12.8.4.1-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cufft_cu12-11.3.3.83-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_curand_cu12-10.3.9.90-py3-none-manylinux_2_27_x86_64.whl.metadat

In [ ]:
#(Optional) CAMeL data (small/light)
!camel_data -i light

No new packages will be installed.


In [ ]:
import os
import math
import numpy as np
import pandas as pd
import regex as re
from collections import Counter

# Split utilities
from sklearn.model_selection import train_test_split

# CAMeL Tools
from camel_tools.disambig.mle import MLEDisambiguator

# Sentence-level embeddings (feature 88)
try:
    from sentence_transformers import SentenceTransformer
    import torch
    HAS_ST = True
except Exception as e:
    print("[warn] sentence-transformers unavailable:", e)
    HAS_ST = False

pd.set_option("display.max_colwidth", 120)


/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


## Config & I/O

In [ ]:
SEED = 42
INPUT_CSV = "/content/drive/MyDrive/Colab Notebooks/data/02_long-clean_phase_2/arabic_generated_abstracts_long_clean_v2.csv"
OUTPUT_DIR = "/content/drive/MyDrive/Colab Notebooks/data/03_phase3_features"
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Load & choose text column

In [ ]:
df = pd.read_csv(INPUT_CSV)

# Prefer 'text_clean' if present; otherwise fall back to 'text'
if "text_clean2" in df.columns:
    TEXT_COL = "text_clean"
elif "text" in df.columns:
    TEXT_COL = "text"
else:
    raise ValueError("No text column found. Expected 'text_clean' or 'text'.")

RAW_COL = None  # set this to 'text_raw' if you have raw text with punctuation/diacritics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Split FIRST (70/15/15), no leakage

In [ ]:
# We stratify if a 'label' column exists (0/1 or string). Otherwise, random split.
stratify_col = df["label"] if "label" in df.columns else None

# Train vs temp (70 / 30)
df_train, df_temp = train_test_split(
    df,
    test_size=0.30,
    random_state=SEED,
    stratify=stratify_col
)

# Val vs Test (15 / 15 of total → 50/50 of the temp)
stratify_temp = df_temp["label"] if "label" in df_temp.columns else None
df_val, df_test = train_test_split(
    df_temp,
    test_size=0.50,
    random_state=SEED,
    stratify=stratify_temp
)

# Mark splits
df_train = df_train.copy(); df_train["split_name"] = "train"
df_val   = df_val.copy();   df_val["split_name"]   = "val"
df_test  = df_test.copy();  df_test["split_name"]  = "test"

print(f"Train: {len(df_train)} | Val: {len(df_val)} | Test: {len(df_test)}")

Train: 29355 | Val: 6290 | Test: 6291


## Arabic utilities

In [ ]:
AR_SENT_SPLIT = re.compile(r"[\.!\?؟…]+")
WS_SPLIT      = re.compile(r"\s+")
TANWEEN_CHARS = set("\u064B\u064C\u064D")         # ً ٌ ٍ
LINK_RE = re.compile(r"https?://\S+")

# CAMeL Disambiguator (MSA default)
mle = MLEDisambiguator.pretrained()

def ar_sentences(text: str):
    if not isinstance(text, str):
        text = "" if text is None else str(text)
    return [s.strip() for s in AR_SENT_SPLIT.split(text) if s.strip()]

def ar_tokens(text: str):
    if not isinstance(text, str):
        text = "" if text is None else str(text)
    toks = [t for t in WS_SPLIT.split(text) if t]
    # Arabic-first tokens (letters/digits/underscore next)
    return [t for t in toks if re.match(r"^\p{Arabic}[\p{Arabic}\p{Nd}_]*$", t)]

def camel_disamb_tokens(tokens):
    """Return list of dicts per token: word, lemma, pos, vox, cas, num, per, gen."""
    out = []
    if not tokens:
        return out
    disamb = mle.disambiguate(tokens)
    for d in disamb:
        if getattr(d, "analyses", None):
            a = d.analyses[0].analysis
            out.append({
                "word": d.word,
                "lemma": a.get("lemma", d.word),
                "pos": a.get("pos", None),  # 'noun', 'verb', 'adj', 'adv', ...
                "vox": a.get("vox", None),  # 'act', 'pass'
                "cas": a.get("cas", None),  # 'nom', 'acc', 'gen'
                "num": a.get("num", None),  # 'sg', 'du', 'pl'
                "per": a.get("per", None),
                "gen": a.get("gen", None),
            })
        else:
            out.append({"word": d.word, "lemma": d.word, "pos": None, "vox": None, "cas": None,
                        "num": None, "per": None, "gen": None})
    return out

## Feature functions (assigned features only)

In [ ]:
from math import log2

def feat_1_total_chars(text:str)->int:
    return len(text if isinstance(text, str) else str(text))

def feat_4_ws_over_C(text:str)->float:
    text = str(text) if not isinstance(text, str) else text
    C = len(text)
    if C == 0:
        return 0.0
    whites = sum(1 for ch in text if ch.isspace())
    return whites / C

def feat_13_hapax_ratio(tokens)->float:
    if not tokens:
        return 0.0
    freq = Counter(tokens)
    hapax = sum(1 for _, c in freq.items() if c == 1)
    return hapax / len(tokens)

def feat_22_entropy_of_word_freq(tokens)->float:
    if not tokens:
        return 0.0
    freq = Counter(tokens)
    N = sum(freq.values())
    H = 0.0
    for c in freq.values():
        p = c / N
        H -= p * log2(p)
    return float(H)

def feat_25_single_quotes(text:str)->int:
    text = str(text) if not isinstance(text, str) else text
    return text.count("'") + text.count("’") + text.count("‘")

def feat_34_total_sentences(sentences)->int:
    return len(sentences)

def feat_43_num_nouns(analyses)->int:
    return sum(1 for a in analyses if (a.get("pos","") or "").lower().startswith("noun"))

def feat_46_num_adverbs(analyses)->int:
    return sum(1 for a in analyses if (a.get("pos","") or "").lower().startswith("adv"))

def feat_55_noun_to_verb_ratio(analyses)->float:
    n_n = sum(1 for a in analyses if (a.get("pos","") or "").lower().startswith("noun"))
    n_v = sum(1 for a in analyses if (a.get("pos","") or "").lower().startswith("verb"))
    if n_v == 0:
        return float("inf") if n_n > 0 else 0.0
    return n_n / n_v

def feat_64_num_nominatives(analyses)->int:
    # case == 'nom' (CAMeL may be sparse without diacritics)
    return sum(1 for a in analyses if (a.get("cas","") or "").lower().startswith("n"))

def feat_67_num_singular_words(analyses)->int:
    return sum(1 for a in analyses if (a.get("num","") or "").lower().startswith("s"))

def feat_76_num_passive_sentences(sentences, analyses_per_sentence)->int:
    cnt = 0
    for anal in analyses_per_sentence:
        has_pass = any((a.get("vox","") or "").lower().startswith("p") for a in anal)
        if has_pass:
            cnt += 1
    return cnt

def feat_85_sent_len_variance(sentences)->float:
    if not sentences:
        return 0.0
    lengths = [len(ar_tokens(s)) for s in sentences]
    return float(np.var(lengths)) if lengths else 0.0

def feat_106_tanween_freq(text_raw_or_clean:str)->int:
    text_raw_or_clean = str(text_raw_or_clean) if not isinstance(text_raw_or_clean, str) else text_raw_or_clean
    return sum(1 for ch in text_raw_or_clean if ch in TANWEEN_CHARS)

def feat_109_link_freq(text:str)->int:
    text = str(text) if not isinstance(text, str) else text
    return len(LINK_RE.findall(text))


## global knobs

In [ ]:
# ---------- Performance knobs ----------
ENABLE_F88 = True    # set False to skip sentence similarity
ENABLE_F97 = True    # set False to skip BERT token-level similarity

# Caps per-doc to keep work bounded
MAX_SENTS_F88 = 12       # only first N sentences per doc for f088
MAX_CHARS_F97 = 400      # only first N chars per doc for f097 tokenization
MAX_TOKS_F97  = 256      # cap subword tokens for f097

# Chunking & checkpoints
CHUNK_SIZE = 2000        # process docs in chunks
CHECKPOINT_DIR = OUTPUT_DIR
os.makedirs(CHECKPOINT_DIR, exist_ok=True)


## Load Models

In [ ]:
# Device for torch models
_device = None
def _torch_device():
    global _device
    if _device is None:
        _device = "cuda" if torch.cuda.is_available() else "cpu"
    return _device

def feat_88_semantic_sim_sentences(sentences, model=None)->float:
    # Fast exits
    if not ENABLE_F88:
        return float("nan")
    if not sentences or len(sentences) < 2:
        return float("nan")
    if model is None:
        return float("nan")

    # Truncate sentence count
    sents = sentences[:MAX_SENTS_F88]

    # Encode on device
    emb = model.encode(
        sents,
        convert_to_tensor=True,
        normalize_embeddings=True,
        device=_torch_device()
    )
    sims = [float(torch.nn.functional.cosine_similarity(emb[i], emb[i+1], dim=0))
            for i in range(len(emb)-1)]
    return float(np.mean(sims)) if sims else float("nan")


from transformers import AutoTokenizer, AutoModel

_BERT_MODEL = None
_TOKENIZER = None

def _load_bert_token_model():
    """Load Arabic BERT model and tokenizer (lazy) and move to device."""
    global _BERT_MODEL, _TOKENIZER
    if _BERT_MODEL is None or _TOKENIZER is None:
        model_name = "aubmindlab/bert-base-arabertv02"
        _TOKENIZER = AutoTokenizer.from_pretrained(model_name)
        _BERT_MODEL = AutoModel.from_pretrained(model_name).to(_torch_device()).eval()
    return _BERT_MODEL, _TOKENIZER

def feat_97_bert_embedding_similarity_tokens(text: str, token_level_model=None) -> float:
    if not ENABLE_F97:
        return float("nan")

    text = str(text or "").strip()
    if len(text) < 2:
        return float("nan")

    # Truncate the raw string to cap cost
    text = text[:MAX_CHARS_F97]

    try:
        model, tokenizer = _load_bert_token_model() if token_level_model is None else token_level_model
        inputs = tokenizer(
            text, return_tensors="pt",
            truncation=True, max_length=MAX_TOKS_F97
        )
        inputs = {k: v.to(_torch_device()) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            hidden = outputs.last_hidden_state.squeeze(0)  # (seq_len, hidden_size)

        if hidden.size(0) < 3:
            return float("nan")

        sims = torch.nn.functional.cosine_similarity(
            hidden[:-1], hidden[1:], dim=1
        ).detach().cpu().numpy()

        return float(np.mean(sims))
    except Exception as e:
        print(f"[warn] BERT token-sim failed: {e}")
        return float("nan")


## Per-document extraction

In [ ]:
def compute_assigned_features_for_text(text, text_raw=None, st_sentence_model=None):
    # Use provided text for both sentence split and tokenization
    sents = ar_sentences(text)
    toks  = ar_tokens(text)
    anal  = camel_disamb_tokens(toks)

    # Analyses per sentence (for passive detection)
    anal_by_sent = [camel_disamb_tokens(ar_tokens(s)) for s in sents]

    # Student 1
    f1   = feat_1_total_chars(text)
    f22  = feat_22_entropy_of_word_freq(toks)
    f43  = feat_43_num_nouns(anal)
    f64  = feat_64_num_nominatives(anal)
    f85  = feat_85_sent_len_variance(sents)
    f106 = feat_106_tanween_freq(text_raw if text_raw is not None else text)

    # Student 2
    f4   = feat_4_ws_over_C(text)
    f25  = feat_25_single_quotes(text)
    f46  = feat_46_num_adverbs(anal)
    f67  = feat_67_num_singular_words(anal)
    f88  = feat_88_semantic_sim_sentences(sents, model=st_sentence_model)
    f109 = feat_109_link_freq(text)

    # Student 3
    f13  = feat_13_hapax_ratio(toks)
    f34  = feat_34_total_sentences(sents)
    f55  = feat_55_noun_to_verb_ratio(anal)
    f76  = feat_76_num_passive_sentences(sents, anal_by_sent)
    f97  = feat_97_bert_embedding_similarity_tokens(text, token_level_model=None)

    return {
        "f001_total_chars": f1,
        "f004_ws_over_C": f4,
        "f013_hapax_ratio": f13,
        "f022_entropy_wordfreq": f22,
        "f025_single_quotes": f25,
        "f034_total_sentences": f34,
        "f043_num_nouns": f43,
        "f046_num_adverbs": f46,
        "f055_noun_to_verb_ratio": f55,
        "f064_num_nominatives": f64,
        "f067_num_singular": f67,
        "f076_num_passive_sentences": f76,
        "f085_sent_len_variance": f85,
        "f088_sem_sim_sentences": f88,
        "f097_bert_sim_tokens": f97,
        "f106_tanween_freq": f106,
        "f109_link_freq": f109
    }


## Apply features per split & Save results

In [ ]:
from pathlib import Path

def apply_features_dataframe_chunked(
    df_split: pd.DataFrame,
    text_col=TEXT_COL,
    raw_col=RAW_COL,
    model=None,
    chunk_size=CHUNK_SIZE,
    split_name="train",
    checkpoint_dir=CHECKPOINT_DIR
) -> pd.DataFrame:
    out_rows = []
    total = len(df_split)
    ckpt_path = Path(checkpoint_dir) / f"phase3_{split_name}_features_ckpt.csv"

    for start in range(0, total, chunk_size):
        end = min(start + chunk_size, total)
        batch = df_split.iloc[start:end]
        rows = []

        for idx, row in batch.iterrows():
            text = row.get(text_col, "")
            text_raw = row.get(raw_col, None) if raw_col else None
            feats = compute_assigned_features_for_text(text, text_raw=text_raw, st_sentence_model=model)
            rows.append(feats)

        feat_df = pd.DataFrame(rows, index=batch.index)
        out_rows.append(pd.concat([batch, feat_df], axis=1))

        # checkpoint this chunk
        pd.concat(out_rows, axis=0).to_csv(ckpt_path, index=False)
        print(f"[{split_name}] processed {end}/{total}")

    result = pd.concat(out_rows, axis=0)
    # remove checkpoint when done
    try:
        Path(ckpt_path).unlink()
    except Exception:
        pass
    return result


# Load sentence model only once (for f088)
st_model = None
if HAS_ST and ENABLE_F88:
    try:
        st_model = SentenceTransformer("Omartificial-Intelligence-Space/Arabic-arabert-all-nli-triplet")
        # pre-warm a tiny encode on device
        _ = st_model.encode(["اختبار"], convert_to_tensor=True, device=_torch_device())
    except Exception as e:
        print("[warn] could not load sentence model:", e)
        st_model = None

# Run with chunking and save final CSVs
train_feats = apply_features_dataframe_chunked(df_train, text_col=TEXT_COL, raw_col=RAW_COL,
                                               model=st_model, split_name="train")
val_feats   = apply_features_dataframe_chunked(df_val,   text_col=TEXT_COL, raw_col=RAW_COL,
                                               model=st_model, split_name="val")
test_feats  = apply_features_dataframe_chunked(df_test,  text_col=TEXT_COL, raw_col=RAW_COL,
                                               model=st_model, split_name="test")

train_csv = os.path.join(OUTPUT_DIR, "phase3_train_features.csv")
val_csv   = os.path.join(OUTPUT_DIR, "phase3_val_features.csv")
test_csv  = os.path.join(OUTPUT_DIR, "phase3_test_features.csv")

train_feats.to_csv(train_csv, index=False)
val_feats.to_csv(val_csv, index=False)
test_feats.to_csv(test_csv, index=False)

print("Saved:")
print(train_csv)
print(val_csv)
print(test_csv)

all_feats = pd.concat([train_feats, val_feats, test_feats], axis=0, ignore_index=True)
all_csv = os.path.join(OUTPUT_DIR, "phase3_all_features_with_split.csv")
all_feats.to_csv(all_csv, index=False)
print("All-in-one CSV:", all_csv)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

[train] processed 2000/29355
[train] processed 4000/29355
[train] processed 6000/29355
[train] processed 8000/29355
[train] processed 10000/29355
[train] processed 12000/29355
[train] processed 14000/29355
[train] processed 16000/29355
[train] processed 18000/29355
[train] processed 20000/29355
[train] processed 22000/29355
[train] processed 24000/29355
[train] processed 26000/29355
[train] processed 28000/29355
[train] processed 29355/29355
[val] processed 2000/6290
[val] processed 4000/6290
[val] processed 6000/6290
[val] processed 6290/6290
[test] processed 2000/6291
[test] processed 4000/6291
[test] processed 6000/6291
[test] processed 6291/6291
Saved:
/content/drive/MyDrive/Colab Notebooks/data/03_phase3_features/phase3_train_features.csv
/content/drive/MyDrive/Colab Notebooks/data/03_phase3_features/phase3_val_features.csv
/content/drive/MyDrive/Colab Notebooks/data/03_phase3_features/phase3_test_features.csv
All-in-one CSV: /content/drive/MyDrive/Colab Notebooks/data/03_phase3_f

In [ ]:
# Optional peek
all_feats.head(10)

,text,text_clean,label,model_name,split_name,f001_total_chars,f004_ws_over_C,f013_hapax_ratio,f022_entropy_wordfreq,f025_single_quotes,...,f046_num_adverbs,f055_noun_to_verb_ratio,f064_num_nominatives,f067_num_singular,f076_num_passive_sentences,f085_sent_len_variance,f088_sem_sim_sentences,f097_bert_sim_tokens,f106_tanween_freq,f109_link_freq
0,إنّ الحضانة من المسائل الحيوية والهامّة في مجال مسائل الأسرة، لاسيما بعد انحلال الزّواج، ومن ثمّة لا إشكال يطرح حينم...,الحضانة المسائل الحيوية والهامة مجال مسائل الاسرة انحلال الزواج اشكال يطرح حينما يكون انحلال الزواج زوجين يحملان الج...,0,human,train,788,0.152284,0.633333,6.022246,0,...,1,4.400000,35,55,1,1722.250000,0.653213,0.405386,0,0
1,يسعى هذا البحث إلى دراسة أوضاع الحجاز في كتابات الرحالة الجزائريين خلال القرن التاسع عشر، من خلال تحليل رحلة الحسين ...,يسعي البحث دراسة اوضاع الحجاز كتابات الرحالة الجزائريين خلال القرن التاسع خلال تحليل رحلة الحسين الورثلاني كتابه نزه...,1,llama,train,720,0.156944,0.519608,5.890323,0,...,0,7.000000,40,63,1,24.160000,0.710701,0.478113,0,0
2,تهدف هذه الدراسة إلى استكشاف وتحليل النقد الإسلامي لاستخدام المناهج الغربية في تفسير النصوص المقدسة، مع التركيز على ...,تهدف الدراسة استكشاف وتحليل النقد الاسلامي لاستخدام المناهج الغربية تفسير النصوص المقدسة التركيز القران الكريم تعتمد...,1,openai,train,850,0.151765,0.640351,6.330397,0,...,0,5.700000,36,76,0,6.560000,0.695090,0.469717,6,0
3,يستكشف هذا البحث تأثير كارل بوبر على الدراسات العربية، مع التركيز على كيفية تأثير فلسفته في الفكر العربي المعاصر. يُ...,يستكشف البحث تاثير كارل بوبر الدراسات العربية التركيز كيفية تاثير فلسفته الفكر العربي المعاصر يستخدم المنهج النقدي ا...,1,llama,train,468,0.160256,0.539683,5.160074,0,...,0,9.000000,21,46,0,5.187500,0.872666,0.493292,2,0
4,يهدف هذا البحث إلى دراسة القياس المحاسبي لبنود الميزانية وفق النظام المحاسبي المالي. لتحقيق هذا الهدف، سيتم استخدام ...,يهدف البحث دراسة القياس المحاسبي لبنود الميزانية وفق النظام المحاسبي المالي لتحقيق الهدف سيتم استخدام منهجية تحليلية...,1,allam,train,476,0.147059,0.681818,5.639017,0,...,1,5.833333,19,40,1,17.250000,0.594534,0.510207,0,0
5,تهدف هذه الدراسة التحليلية إلى استكشاف العلاقة المعقدة بين الإجرام المعلوماتي والجرائم الاقتصادية والمالية على الصعي...,تهدف الدراسة التحليلية استكشاف العلاقة المعقدة الاجرام المعلوماتي والجرائم الاقتصادية والمالية الصعيد الدولي تتناول ...,1,openai,train,1049,0.149666,0.682759,6.611878,0,...,1,3.333333,58,95,1,28.204082,0.741228,0.496925,1,0
6,تتناول هذه الدراسة تقييم وتحليل تنافسية قطاع الصناعة التحويلية الجزائرية ومقارنته مع الصناعات التحويلية لدول المغرب ...,تتناول الدراسة تقييم وتحليل تنافسية قطاع الصناعة التحويلية الجزائرية ومقارنته الصناعات التحويلية لدول المغرب العربي ...,0,human,train,986,0.153144,0.685714,6.622898,0,...,1,3.500000,51,90,1,0.000000,NaN,0.489060,6,0
7,أدى ظهور نظام التقاضي الإلكتروني، الذي يعد نتاجاً للثورة المعلوماتية والتقدم التكنولوجي، إلى دفع العديد من التشريعات...,ادي ظهور نظام التقاضي الالكتروني يعد نتاجا للثورة المعلوماتية والتقدم التكنولوجي دفع العديد التشريعات الدولية والداخ...,1,jais,train,493,0.146045,0.827586,5.685567,0,...,0,4.125000,18,42,0,4.250000,0.521520,0.454112,1,0
8,تهدف هذه الورقة البحثية إلى استكشاف مسؤولية الولي عن الأفعال الضارة التي يرتكبها أبناؤه القصر وفقًا لمقتضيات القانون...,تهدف الورقة البحثية استكشاف مسؤولية الولي الافعال الضارة يرتكبها ابناؤه القصر وفقا لمقتضيات القانون المدني وقانون ال...,1,openai,train,777,0.151866,0.684685,6.391219,0,...,0,6.400000,30,73,0,40.960000,0.535796,0.490265,1,0
9,تهدف الدراسة إلى تقييم تأثير برنامج تدريبي مقترح يعتمد على أسلوب التدريب الفتري في تطوير بعض المتغيرات الوظيفية لدى ...,تهدف الدراسة تقييم تاثير برنامج تدريبي مقترح يعتمد اسلوب التدريب الفتري تطوير المتغيرات الوظيفية لاعبي كرة القدم اعت...,1,openai,train,876,0.147260,0.678571,6.302629,0,...,0,5.583333,33,77,0,9.714286,0.587257,0.498078,5,0
